In [3]:
import cv2
import numpy as np

In [2]:
# 가우시안 필터 -> 노이즈 제거
def del_noise(input_mat):
    ret_mat = cv2.GaussianBlur(input_mat, (3,3), 0)

    return ret_mat

In [3]:
# grayscale로 변환, 이진화

def make_gray_and_binary(input_mat):
    gray_mat = cv2.cvtColor(input_mat, cv2.COLOR_BGR2GRAY)
    boundary_value, ret_mat = cv2.threshold(gray_mat, 127, 255, cv2.THRESH_BINARY)
    
    return ret_mat

In [4]:
# 모폴로지(열림)

def morphologing(input_mat):
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    ret_mat = cv2.morphologyEx(input_mat, cv2.MORPH_OPEN, k)

    return ret_mat

In [5]:
# 엣지 검출(라플라시안 필터 or 캐니)

def detect_edge(input_mat, lapla_or_canny):
    if(lapla_or_canny == 0):
        ret_mat = cv2.Laplacian(input_mat, -1)
    elif(lapla_or_canny == 1):
        ret_mat = cv2.Canny(input_mat, 100, 200)
    
    return ret_mat

In [7]:
def extract_obj(input_mat):
    copy = input_mat.copy()

    # grayscale 변환
    gray = cv2.cvtColor(input_mat, cv2.COLOR_BGR2GRAY)
    # 가우시안 블러링
    gray_blur = cv2.GaussianBlur(gray, (3,3), 0)
    # canny 필터
    edged = cv2.Canny(gray_blur, 30, 220)

    # edge 검출 이미지
    # cv2.imshow('canny_edged', edged)
    # cv2.waitKey()

    # 컨투어 리스트 획득
    cnts, hr = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[:]

    # 영역이 최대인 rect 획득
    x, y, w, h = cv2.boundingRect(cnts[0])
    
    for c in cnts:
        a, b, c, d = cv2.boundingRect(c)
        if(c*d > w*h):
            x, y, w, h = a, b, c, d

    rect = (x, y, x + w, y + h)

    # 선택된 rect 확인
    # rect_draw = cv2.rectangle(input_mat.copy(), rect, (255, 0, 0))
    # cv2.imshow('rect_draw', rect_draw)
    # cv2.waitKey()

    mask = np.zeros(input_mat.shape[:2], dtype= np.uint8)
    cv2.grabCut(input_mat, mask, rect, None, None, 1, cv2.
    GC_INIT_WITH_RECT)

    mask_dst = np.where((mask == 0)|(mask == 2), 0, 1).astype('uint8')

    dst = input_mat*mask_dst[:, :, np.newaxis]

    # 출력 확인
    # cv2.imshow('dst', dst)
    # cv2.waitKey()
    # cv2.destroyAllWindows()

    return dst